In [1]:
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.cm
import sys
sys.path.insert(0,'../')
%load_ext autoreload
%autoreload 2
from utils import citibike_helpers
from utils.gspan import gSpan
import numpy as np
import pandas as pd  #requirement comes with anaconda
import datetime 
from datetime import datetime as dt
import seaborn as sns
import operator
import networkx as nx
from datetime import timedelta

import community #requires separate install -  pip install python-louvain
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [2]:
inputfile="../datasets/citibike/201701-citibike-tripdata.csv.gz"
df = citibike_helpers.load_citibike_data(inputfile)
df = citibike_helpers.calculate_trip_durations_citibike(df)
#Here we focus on a subset of "interesting" trips with duration between 5 and 120 minutes.
subset_trips=df[(df['Trip Duration Minutes']>5) & (df['Trip Duration Minutes']<120)]
subset_trips.sort_values('Start Time',inplace=True)
subset_trips['Start Time'] = pd.DatetimeIndex(subset_trips['Start Time'])
subset_trips.head(2)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Trip Duration Minutes
0,00:11:20,2017-01-01 00:00:21,2017-01-01 00:11:41,3226,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,40.775794,-73.976206,25542,Subscriber,1965.0,2,11.333333
1,00:21:23,2017-01-01 00:00:45,2017-01-01 00:22:08,3263,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,40.748549,-73.988084,21136,Subscriber,1987.0,2,21.383333


In [3]:
unique_start_stations=citibike_helpers.get_unique_column_values(subset_trips,'Start Station ID')
unique_end_stations=citibike_helpers.get_unique_column_values(subset_trips,'End Station ID')
station_ids=set(unique_start_stations).union(set(unique_end_stations))
station2id = {v:k for k,v in enumerate(station_ids)}

In [9]:
directed = False
graph_number = 0
time = 24
start_time = pd.DatetimeIndex(['2017-01-01T00:00:00.000000000'])
end_time = pd.DatetimeIndex(['2017-02-01T00:00:00.000000000'])
next_time = start_time + timedelta(hours=time)
if directed:
    g = nx.DiGraph()
else:
    g = nx.Graph()
all_graphs = []
while next_time < end_time:
    fl = subset_trips[(subset_trips['Start Time'].values > start_time.values) & (subset_trips['Start Time'].values <= next_time.values)]
    for sts,ens in zip(fl['Start Station ID'].values,fl['End Station ID'].values):
        if (sts,ens) in g.edges():
            g[sts][ens]['weight'] = g[sts][ens]['weight']+1
        else:
            g.add_edge(sts,ens,weight=1)

    node2id = {station2id[v]:k for k,v in enumerate(g.nodes())}
    start_time = next_time
    next_time = start_time + timedelta(hours=time)
    graph_number += 1
    all_graphs.append(g.copy())
    nx.write_edgelist(g,path='../datasets/citibike/edgelist_24.txt',data=False,delimiter=',')
    g.clear()
    break